Finetune a Pretrained Hugginface model

In [ ]:
#Import Libraries
import os

#Data manipulation
import pandas as pd
import numpy as np
!pip install datasets
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords

#finetuning
!pip install transformers
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer

#Evaluation
from datasets import load_metric

from huggingface_hub import notebook_login
!pip install transformers[torch]
!pip install evaluate
import evaluate






In [ ]:
!pip install accelerate -U

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#login to huggingface
notebook_login()

Activate weight and Biases

In [ ]:
os.environ["WANDB_DISABLED"] = "false"

Load The Datasets

In [ ]:
#load data
train_df=pd.read_csv('/content/drive/MyDrive/Azubi datasets/dataset/Train.csv')

#check first 5

train_df.head()


,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


AS Noticed in EDA we will drop the rows with missing values

In [ ]:
#Drop null rows
train_df.dropna(axis=0,inplace=True)

#check for the presence of null values again
train_df.isna().sum()


tweet_id     0
safe_text    0
label        0
agreement    0
dtype: int64

Remove stopwords

In [ ]:
nltk.download('stopwords')

stop_words = stopwords.words('english')
train_df['safe_text'] = train_df['safe_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Split the Train the dataset into training(for learning ) and evaluation(For computing metrics to capture overfitting) sets

In [ ]:
train , eval = train_test_split(train_df,train_size=0.8,stratify=train_df['label'])

check train data

In [ ]:
train.head()

,tweet_id,safe_text,label,agreement
367,FTJI39IG,Disney Tie Hides Biggest Measles Risk Adults C...,0.0,1.000000
1700,071DYIT4,"“<user> ""The benefits measles"" RT <user> How r...",-1.0,0.333333
5700,B9OTD916,"I know bigger morons, ones refuse get measles ...",1.0,1.000000
9236,SSPRHTIV,"California lets parents object vaccination ""ph...",0.0,1.000000
9600,2IGZJWZ6,5 New Cases Of Disneyland-Linked Measles Repor...,0.0,1.000000


Check eval

In [ ]:
eval.head()

,tweet_id,safe_text,label,agreement
6492,G0Y9Z673,#Vaxxed Very interesting #documentary! #Vaccin...,-1.0,0.333333
4600,1UNPH66S,Children's vaccines restricted county health d...,-1.0,0.666667
4906,S6P1U76J,AM-News : Child immunization clinic Tuesday Wi...,1.0,0.666667
5186,EHKZ9UWZ,AIDS N : Study: Thousands Vaccinated Teens Bor...,0.0,0.666667
5423,SDML9PUM,CDC Says Ebola Vaccine Only Works White People...,-1.0,0.666667


Check the shapes of both dataset

In [ ]:
print(f'Shape of training set: {train.shape}\nShape of evaluation set: {eval.shape}')

Shape of training set: (7999, 4)
Shape of evaluation set: (2000, 4)


Save the splitted data to be loaded as huggingface dataset

In [ ]:
train.to_csv('/content/drive/MyDrive/Azubi datasets/dataset/Train_subset.csv',index=False)
eval.to_csv('/content/drive/MyDrive/Azubi datasets/dataset/eval_subset.csv',index=False)

In [ ]:
datasets=load_dataset('csv',data_files={'train':'/content/drive/MyDrive/Azubi datasets/dataset/Train_subset.csv','eval':'/content/drive/MyDrive/Azubi datasets/dataset/eval_subset.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1c285aa253853d36/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Create a function to tokenize the data
create a function to transform the labels or call lable encoder


In [ ]:
#Create a tokenizer instance
tokenizer=AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
#function to tokenize text
def tokenize_data(data):
    return tokenizer(data['safe_text'],padding='max_length')


#function to transform labels
def transfom_label(data):

    #extract label
    label=data['label']
    num=0
    #create contitions
    if label==-1: #Negative
        num=0
    elif label==0: #Neutral
        num=1
    else:
        num=2      #Positive
    return {'labels':num}



Use .map to apply function to dataset, .map speeds up data processing

In [ ]:
#The following columns will be removed after tokenization
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']

#Tokenize the text data
datasets=datasets.map(tokenize_data,batched=True)

#transform the labels
datasets=datasets.map(transfom_label,remove_columns=remove_columns)

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7999 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

# Training a Bert model


Extract the train and eval datasets from datasets

In [ ]:
#extract train datasets
train_dataset=datasets['train'].shuffle(seed=0)
#extract eval datasets
eval_dataset=datasets['eval'].shuffle(seed=0)

In [ ]:
def compute_metrics(eval_preds):
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    f1_metric = load_metric("f1")

    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    precision=precision_metric.compute(predictions=predictions, references=labels,average='weighted')
    recall= recall_metric.compute(predictions=predictions, references=labels,average='weighted')
    f1=f1_metric.compute(predictions=predictions, references=labels,average='weighted')
    return f1
#set the training arguments
trainargs=TrainingArguments('tweet_sentiments_analysis_distilbert-base-uncased',num_train_epochs=7,evaluation_strategy="epoch",save_strategy='epoch',load_best_model_at_end=True,push_to_hub=True)
#model
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)

#trainerinstance
trainer = Trainer(
    model,
    trainargs,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Cloning https://huggingface.co/Junr-syl/tweet_sentiments_analysis_distilbert-base-uncased into local empty directory.


In [ ]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.754200,0.742939,0.663047
2,0.726400,0.721375,0.678193
3,0.637600,0.660979,0.717085
4,0.519600,0.757809,0.729057
5,0.434400,0.867036,0.724755
6,0.334200,1.052164,0.722311
7,0.284100,1.171507,0.717982


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=7000, training_loss=0.5317181396484375, metrics={'train_runtime': 2986.8103, 'train_samples_per_second': 18.747, 'train_steps_per_second': 2.344, 'total_flos': 7417379329016832.0, 'train_loss': 0.5317181396484375, 'epoch': 7.0})

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Jul19_16-21-33_1f5dc6195aad/events.out.tfevents.1689783704.1f5dc6195aad.1861.3:   0%|        …

To https://huggingface.co/Junr-syl/tweet_sentiments_analysis_distilbert-base-uncased
   bbd3985..4c2d3c5  main -> main

   bbd3985..4c2d3c5  main -> main

To https://huggingface.co/Junr-syl/tweet_sentiments_analysis_distilbert-base-uncased
   4c2d3c5..98672c3  main -> main

   4c2d3c5..98672c3  main -> main



'https://huggingface.co/Junr-syl/tweet_sentiments_analysis_distilbert-base-uncased/commit/4c2d3c5e5ee8fb9af35081158953b1e245707eb7'